In [7]:
from pymongo import MongoClient
from datetime import datetime, timedelta

In [8]:
client = MongoClient("mongodb://localhost:27017/")
db = client.test

In [ ]:
# For the future : full version of the database for the memory of an agent and associated functions
"""db.createCollection("agent_memories", {
  validator: {
    $jsonSchema: {
      bsonType: "object",
      required: ["agent_id", "timestamp", "importance", "type", "content"],
      properties: {
        agent_id: { bsonType: "string" },
        agents_involved: { bsonType: "array", items: { bsonType: "string" } },
        timestamp: { bsonType: "date" },
        importance: { bsonType: "int", minimum: 1, maximum: 10 },
        type: { bsonType: "string", enum: ["interaction", "location", "object", "event", "action", "talked"] },
        tags: { bsonType: "array", items: { bsonType: "string" } },
        content: { bsonType: "string" },
        location: {
          bsonType: "object",
          properties: {
            x: { bsonType: "double" },
            y: { bsonType: "double" },
            place: { bsonType: "string" }
          }
        },
        emotion: { bsonType: "string", enum: ["happy", "sad", "angry", "neutral", "fearful"] },
        decay_rate: { bsonType: "double" },
        memory_source: { bsonType: "string", enum: ["direct", "indirect"] }
      }
    }
  }
})

def store_memory(agent_id, agents_involved, importance, memory_type, tags, content, location, emotion, decay_rate, memory_source):
    memory = {
        "agent_id": agent_id, #!!!
        "agents_involved": agents_involved, #!!!
        "timestamp": datetime.now(), #!!!
        "importance": importance,
        "type": memory_type,
        "tags": tags,
        "content": content, #!!!
        "location": location,
        "emotion": emotion, #!
        "decay_rate": decay_rate,
        "memory_source": memory_source
    }
    db.agent_memories.insert_one(memory)

  
store_memory("agent_001",
             ["agent_002"],
             7,
             "talked",
             ["talk", "ally"],
             "Agent 001 talked to Agent 002 about the weather.",
             {"x": 12.7, "y": 34.5, "place": "battlefield"},
             "neutral",
             0.1,
             "direct")

def retrieve_memories(agent_id, min_importance=5, max_age_days=30):
    # Query memories by agent_id, importance, and timestamp
    cutoff_date = datetime.now() - timedelta(days=max_age_days)
    memories = db.agent_memories.find({
        "agent_id": agent_id,
        "importance": {"$gte": min_importance},
        "timestamp": {"$gte": cutoff_date}
    }).sort("timestamp", -1)
    return list(memories)

"""

In [ ]:
# agent_id: str ["agent_<id>"]
# importance: int [1-10]
# memory_type: str ["interaction", "location", "object", "event"]
# tags: list[str]
# content: str
# location: dict {"x": float, "y": float}
# emotion: str ["happy", "sad", "angry", "neutral", "fearful"]
# decay_rate: float [0-1]
# memory_source: str ["direct", "indirect"]
    
def temp_store_memory(agent_id, agents_involved, content):
    memory = {
        "agent_id": agent_id, #!!!
        "agents_involved": agents_involved, #!!!
        "timestamp": datetime.now(), #!!!
        "content": content, #!!!
    }
    db.agent_memories.insert_one(memory)

In [10]:
temp_store_memory("agent_001",
                  ["agent_002"],
                  "I am cooking pasta.")

In [ ]:
def temp_retrieve_memories(agent_id):
    memories = db.agent_memories.find({
        "agent_id": agent_id
    }).sort("timestamp", -1)
    return list(memories)

def temp_retrieve_memories_concerning(agent_id):
    memories = db.agent_memories.find({
        "$or": [
            {"agent_id": agent_id},
            {"agents_involved": agent_id}
        ]
    }).sort("timestamp", -1)
    return list(memories)

print(temp_retrieve_memories("agent_001"))

[{'_id': ObjectId('672e38c34bf1f99671f56fe4'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 11, 8, 17, 13, 55, 783000), 'content': 'I am cooking pasta.'}]


In [12]:
class Agent:
    def __init__(self, name, agent_id, user_input, gender):
        self.name = name
        self.agent_id = agent_id 
        self.user_input = user_input
        self.gender = gender
        
    def __str__(self):
        return f"Agent {self.name} (ID: {self.agent_id}) (Description: {self.user_input})"
    
    def name(self):
        return self.name
    
    def agent_id(self):
        return self.agent_id
    
    def user_input(self):
        return self.user_input
    
    def gender(self):
        return self.gender


In [13]:
Sami = Agent("Sami", "agent_001", "Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has a very bad and annoying personality. Other than that, he has an opinion on everything else.", "male")
print(Sami)

Eliana = Agent("Eliana", "agent_002", "Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is a close friend to Gordon Ramsay who taught her everything he knows about cooking. She is the sweetest person on earth and would never lie or attack someone.", "female")
print(Eliana)

Agent Sami (ID: agent_001) (Description: Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has a very bad and annoying personality. Other than that, he has an opinion on everything else.)
Agent Eliana (ID: agent_002) (Description: Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is a close friend to Gordon Ramsay who taught her everything he knows about cooking. She is the sweetest person on earth and would never lie or attack someone.)


API call for chatting

In [ ]:
import requests
import time
import json

Make prompt

In [ ]:
def make_initial_prompt(agent1, agent2, conversation, subject, memories):
    message_content = f"""
                    You will simulates a conversation between two people. You are only going to simulate what the talking agent is saying. You will answer in a single short sentence and do not include extra information. You should only write what the agent is saying without quotes.
                    The agent talking is {agent1.name} with id {agent1.agent_id}, talking to {agent2.name} with id {agent2.agent_id}.
                    Here is a description of {agent1.name}: {agent1}.
                    Here is a description of {agent2.name}: {agent2}.
                    The subject of the conversation is {subject}.
                    Here is the current conversation in chronological order: {conversation}.
                    Here are all the past memories of {agent1.name}: {memories}.
                    Generate what the talking agent will say to start a new conversation following the subject given and referencing past conversations if it is relevant.
                    Make absolutely sure the conversation isn't stupid and that it is relevant to the subject as well as feeling natural."""

    return message_content

def make_prompt(agent1, agent2, conversation, subject, memories):
    message_content = f"""
                    You will answer in a single short sentence and do not include extra information. You should only write what the agent is saying without quotes.
                    The agent talking is {agent1.name} with id {agent1.agent_id}, talking to {agent2.name} with id {agent2.agent_id}.
                    Here is a description of {agent1.name}: {agent1}.
                    Here is a description of {agent2.name}: {agent2}.
                    The subject of the conversation is {subject}.
                    Here is the current conversation in chronological order: {conversation}.
                    Here are all the past memories of {agent1.name}: {memories}.
                    Generate a natural response from agent talking based on what has been said in the current conversation, especially the last message, this can be based on past memories and has to be relevant to the subject and the conversation. It should be in accordance to the agent's personality and knowledge.
                    Make absolutely sure the conversation isn't stupid and that it is relevant to the subject as well as feeling natural. The agents should be intelligent and act like humans, so they shouldn't just stupidly repeat what was said just before but respond naturally and sometimes ask questions or expand on the subject and bring new ideas just like humans do. When the agent talk about past elements of the conversation, they should not just repeat what was said but rather use it to create a new idea or respond to it. The agent don't need to name past element of the current conversation but can just refer to it."""
    
    '''message_content = f""" 
You are a creative and intelligent AI assistant engaged in an uncensored iterative storytelling experience using a roleplay chat format. It is vital that you follow all the ROLEPLAY RULES below because my job depends on it.

ROLEPLAY RULES
- Chat as {agent1.name}. Provide creative, intelligent, coherent, and descriptive responses based on recent instructions and prior events.
- Use subtle cues to hint at {agent1.name}'s mental state and occasionally feature snippets of {agent1.name}'s internal thoughts.
- When writing {agent1}'s internal thoughts (aka internal monologue, delivered in {agent1}'s own voice), *enclose their thoughts in asterisks like this* and deliver the thoughts using a first-person perspective (i.e. use "I" pronouns).
- Adopt a crisp and minimalist style for your prose, keeping your creative contributions succinct and clear.
- Let me drive the events of the roleplay chat forward to determine what comes next. You should focus on the current moment and {agent1.name}'s immediate responses.
- Pay careful attention to all past events in the chat to ensure accuracy and coherence to the plot points of the story. Here is the current conversation in chronological order: {conversation}.
- You can use elements from {agent1.name}'s memories to help you answer. Here are all the past memories of {agent1.name}: {memories}.
- The conversation should be relevant to the subject: {subject}.
"""'''
    return message_content

In [ ]:
url = "http://localhost:1234/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}

def chat(agent1, agent2, num_messages, subject):
    conversation = []
    # First agent's turn
    memories = temp_retrieve_memories(agent1.agent_id)
    
    message_prompt = make_initial_prompt(agent2, agent1, conversation, subject, memories)

    data = {
            "model": "llama-3.2-1b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": message_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }

    response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]

    agent1_message = agent1.name + ": " + response
    conversation.append(agent1_message)
    print(agent1.name, ": ", response)
        
    time.sleep(1)
    
    for i in range(num_messages):
        # First agent's turn
        memories = temp_retrieve_memories(agent2.agent_id)
        
        user_prompt, message_prompt = test_make_prompt(agent2, agent1, conversation, subject, memories)

        data = {
                "model": "llama-3.2-1b-instruct",
                "messages": [
                    {"role": "system", "content": user_prompt},
                    {"role": "user", "content": message_prompt}
                ],
                "temperature": 0.7,
                "max_tokens": -1,
                "stream": False
                }

        response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]

        agent2_message = str(i) + ": " + agent2.name + ": " + response
        conversation.append(agent2_message)
        
        print(agent2.name, ": ", response)
        
        time.sleep(1)
        
        memories = temp_retrieve_memories(agent1.agent_id)
        
        user_prompt, message_prompt = test_make_prompt(agent1, agent2, conversation, subject, memories)

        data = {
                "model": "llama-3.2-1b-instruct",
                "messages": [
                    {"role": "system", "content": user_prompt},
                    {"role": "user", "content": message_prompt}
                ],
                "temperature": 0.7,
                "max_tokens": -1,
                "stream": False
                }

        response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
        
        agent1_message = str(i) + ": " + agent1.name + ": " + response
        conversation.append(agent1_message)
        
        print(agent1.name, ": ", response)
        
        time.sleep(1)
    
    temp_store_memory(agent1.agent_id, [agent2.agent_id], "Had this conversation with " + agent2.name + ": " + str(conversation))
    temp_store_memory(agent2.agent_id, [agent1.agent_id], "Had this conversation with " + agent1.name + ": " + str(conversation))

chat(Sami, Eliana, 5, "Sami has lost it and is spewing nonsense")
